In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Импортируем данные

In [244]:
train = pd.read_parquet('train.parquet')
train

,target,order_create_time,session_end_time,model_1,model_2,model_3,model_4,model_5
0,1,1975-12-26 09:30:08,1975-12-26 09:21:03.393,0.798127,0.644805,0.039843,1,0.430339
1,0,1975-12-26 10:28:57,1975-12-26 11:01:03.407,-0.573826,0.220081,0.020889,0,0.177593
2,1,1975-12-25 11:24:28,1975-12-25 11:23:12.517,-0.665887,0.644805,0.039810,0,0.431897
3,1,1975-12-26 14:16:06,1975-12-26 14:29:57.153,-0.284625,0.644805,0.040026,0,0.432045
4,0,1975-12-26 18:21:47,1975-12-26 18:17:54.257,-1.710108,-0.221541,0.020585,0,0.174115
...,...,...,...,...,...,...,...,...
104590,1,1975-12-16 12:11:32,1975-12-16 12:56:06.990,-0.784541,0.220081,0.020901,0,0.188639
104591,1,1975-11-08 11:29:50,1975-11-07 23:55:00.773,-0.504846,0.220081,0.029831,0,0.175063
104592,0,1975-11-12 09:14:35,1975-11-12 09:15:26.047,-1.710108,-0.221541,0.021167,0,0.191408
104593,1,1975-11-10 17:28:07,1975-11-10 17:23:02.973,-0.403279,0.644805,0.040814,0,0.452594


In [243]:
test = pd.read_parquet('test.parquet')
test

,order_id,order_create_time,session_end_time,model_1,model_2,model_3,model_4,model_5
0,1350922,1976-02-05 15:08:37,1976-02-05 15:04:28.867,-1.710108,0.203183,0.040015,1,0.442343
1,1354989,1976-02-07 15:26:00,1976-02-07 15:27:26.593,0.086020,0.220081,0.021101,1,0.191027
2,1352637,1976-02-06 11:43:58,1976-02-05 18:34:59.477,-0.438741,0.644805,0.066312,0,0.400509
3,1350050,1976-02-05 11:26:19,1976-02-05 11:09:28.247,-1.395752,0.203183,0.083039,1,0.414590
4,1341733,1976-02-01 19:36:32,1976-02-01 09:53:54.103,-0.441443,0.644805,0.052902,0,0.411544
...,...,...,...,...,...,...,...,...
17191,1358397,1976-02-09 16:13:20,1976-02-02 09:02:14.140,-0.429658,0.644805,0.908250,0,0.196781
17192,1357968,1976-02-09 14:30:59,1976-02-09 14:26:24.480,-1.710108,0.203183,0.040665,0,0.411543
17193,1358835,1976-02-09 18:39:45,1976-02-09 18:30:36.407,0.086020,0.644805,0.039744,1,0.426308
17194,1365692,1976-02-12 17:50:18,1976-02-12 16:13:15.153,-1.710108,0.203183,0.041701,0,0.433750


В качестве признаков будем использовать предсказания моделей, разницу между моментами создания заказа и закрытия сессии (назовём этот признак `time_delta`), а также кластеризуем `time_delta` и возьмём в качестве дополнительных признаков номер кластера и расстояние до центра кластера (идея с кластеризацией взята из ноутбука, который Stephan Kafanov выложил в материалы).

In [245]:
from sklearn.cluster import KMeans

train['time_delta'] = (train['order_create_time'] - train['session_end_time']).dt.total_seconds()
test['time_delta'] = (test['order_create_time'] - test['session_end_time']).dt.total_seconds()

kmeans = KMeans(n_clusters=8, max_iter=500, n_init=100)

no_nans_train = train[~train.time_delta.isna()]
train_clusters = kmeans.fit_predict(no_nans_train[['time_delta']])
train.loc[~train.time_delta.isna(), 'clusters'] = train_clusters

clust_centers = kmeans.cluster_centers_

no_nans_test = test[~test.time_delta.isna()]
test_clusters = kmeans.predict(no_nans_test[['time_delta']])
test.loc[~test.time_delta.isna(), 'clusters'] = test_clusters

for i in range(8):
    idx = train['clusters'] == i
    train.loc[idx, 'dist'] = (train.loc[idx, 'time_delta'] - clust_centers[i]).abs()
    idx = test['clusters'] == i
    test.loc[idx, 'dist'] = (test.loc[idx, 'time_delta'] - clust_centers[i]).abs()

In [246]:
train

,target,order_create_time,session_end_time,model_1,model_2,model_3,model_4,model_5,time_delta,clusters,dist
0,1,1975-12-26 09:30:08,1975-12-26 09:21:03.393,0.798127,0.644805,0.039843,1,0.430339,544.607,0.0,543.276013
1,0,1975-12-26 10:28:57,1975-12-26 11:01:03.407,-0.573826,0.220081,0.020889,0,0.177593,-1926.407,0.0,3014.290013
2,1,1975-12-25 11:24:28,1975-12-25 11:23:12.517,-0.665887,0.644805,0.039810,0,0.431897,75.483,0.0,1012.400013
3,1,1975-12-26 14:16:06,1975-12-26 14:29:57.153,-0.284625,0.644805,0.040026,0,0.432045,-831.153,0.0,1919.036013
4,0,1975-12-26 18:21:47,1975-12-26 18:17:54.257,-1.710108,-0.221541,0.020585,0,0.174115,232.743,0.0,855.140013
...,...,...,...,...,...,...,...,...,...,...,...
104590,1,1975-12-16 12:11:32,1975-12-16 12:56:06.990,-0.784541,0.220081,0.020901,0,0.188639,-2674.990,0.0,3762.873013
104591,1,1975-11-08 11:29:50,1975-11-07 23:55:00.773,-0.504846,0.220081,0.029831,0,0.175063,41689.227,4.0,27790.947360
104592,0,1975-11-12 09:14:35,1975-11-12 09:15:26.047,-1.710108,-0.221541,0.021167,0,0.191408,-51.047,0.0,1138.930013
104593,1,1975-11-10 17:28:07,1975-11-10 17:23:02.973,-0.403279,0.644805,0.040814,0,0.452594,304.027,0.0,783.856013


In [247]:
test

,order_id,order_create_time,session_end_time,model_1,model_2,model_3,model_4,model_5,time_delta,clusters,dist
0,1350922,1976-02-05 15:08:37,1976-02-05 15:04:28.867,-1.710108,0.203183,0.040015,1,0.442343,248.133,0.0,839.750013
1,1354989,1976-02-07 15:26:00,1976-02-07 15:27:26.593,0.086020,0.220081,0.021101,1,0.191027,-86.593,0.0,1174.476013
2,1352637,1976-02-06 11:43:58,1976-02-05 18:34:59.477,-0.438741,0.644805,0.066312,0,0.400509,61738.523,4.0,7741.651360
3,1350050,1976-02-05 11:26:19,1976-02-05 11:09:28.247,-1.395752,0.203183,0.083039,1,0.414590,1010.753,0.0,77.130013
4,1341733,1976-02-01 19:36:32,1976-02-01 09:53:54.103,-0.441443,0.644805,0.052902,0,0.411544,34957.897,0.0,33870.013987
...,...,...,...,...,...,...,...,...,...,...,...
17191,1358397,1976-02-09 16:13:20,1976-02-02 09:02:14.140,-0.429658,0.644805,0.908250,0,0.196781,630665.860,3.0,26647.800606
17192,1357968,1976-02-09 14:30:59,1976-02-09 14:26:24.480,-1.710108,0.203183,0.040665,0,0.411543,274.520,0.0,813.363013
17193,1358835,1976-02-09 18:39:45,1976-02-09 18:30:36.407,0.086020,0.644805,0.039744,1,0.426308,548.593,0.0,539.290013
17194,1365692,1976-02-12 17:50:18,1976-02-12 16:13:15.153,-1.710108,0.203183,0.041701,0,0.433750,5822.847,0.0,4734.963987


In [214]:
new_X_train = train.drop(['order_create_time', 'session_end_time', 'target'], axis=1)
y_train = train.target

Теперь подберём лучшие параметры с помощью `optuna` на кросс-валидацией с early-stopping по ROC AUC.

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np

def objective(trial):
    params = {
        "objective": "binary",
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 1, log=True),
        "metric": "auc"
    }

    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_idx, valid_idx in kf.split(new_X_train):
        X_train_fold, X_valid_fold = new_X_train.iloc[train_idx], new_X_train.iloc[valid_idx]
        y_train_fold, y_valid_fold = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        lgb_train = lgb.Dataset(
            X_train_fold, 
            label=y_train_fold,
            categorical_feature=['model_4', 'clusters']
        )
        lgb_valid = lgb.Dataset(
            X_valid_fold, 
            label=y_valid_fold,
            reference=lgb_train
        )

        model = lgb.train(
            params=params,
            train_set=lgb_train,
            valid_sets=[lgb_valid],
            valid_names=['valid'],
            callbacks=[
                lgb.log_evaluation(period=20),
                lgb.early_stopping(stopping_rounds=50, verbose=True)
            ],
            num_boost_round=500
        )

        valid_preds = model.predict(X_valid_fold)
        metric_value = roc_auc_score(np.array(y_valid_fold), valid_preds)
        scores.append(metric_value)

    return np.mean(scores)

In [238]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Лучшие параметры:", study.best_params)

[I 2025-05-25 21:08:13,761] A new study created in memory with name: no-name-6a59edd7-d88e-47e1-89c5-3c200a2d9aa2


Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.949504
[40]	valid's auc: 0.949943
[60]	valid's auc: 0.950278
[80]	valid's auc: 0.950592
[100]	valid's auc: 0.951953
[120]	valid's auc: 0.952293
[140]	valid's auc: 0.95234
[160]	valid's auc: 0.952401
[180]	valid's auc: 0.952671
[200]	valid's auc: 0.952927
[220]	valid's auc: 0.95301
[240]	valid's auc: 0.953117
[260]	valid's auc: 0.953265
[280]	valid's auc: 0.953341
[300]	valid's auc: 0.953434
[320]	valid's auc: 0.953473
[340]	valid's auc: 0.953597
[360]	valid's auc: 0.953704
[380]	valid's auc: 0.953753
[400]	valid's auc: 0.953859
[420]	valid's auc: 0.953988
[440]	valid's auc: 0.954067
[460]	valid's auc: 0.954171
[480]	valid's auc: 0.954205
[500]	valid's auc: 0.954399
Did not meet early stopping. Best iteration is:
[498]	valid's auc: 0.9544
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.950938
[40]	valid's auc: 0.951579
[60]	valid's auc: 0.951721
[80]	valid's auc: 0.951838
[1

[I 2025-05-25 21:08:24,714] Trial 0 finished with value: 0.9529426380283518 and parameters: {'num_leaves': 16, 'learning_rate': 0.007125953465932265}. Best is trial 0 with value: 0.9529426380283518.


Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.955365
[40]	valid's auc: 0.955563
[60]	valid's auc: 0.955272
Early stopping, best iteration is:
[29]	valid's auc: 0.955865
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.954779
[40]	valid's auc: 0.954822
[60]	valid's auc: 0.954297
Early stopping, best iteration is:
[25]	valid's auc: 0.954925
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.951714
[40]	valid's auc: 0.951423
[60]	valid's auc: 0.951071
Early stopping, best iteration is:
[26]	valid's auc: 0.951972
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.954742
[40]	valid's auc: 0.955012
[60]	valid's auc: 0.954953
[80]	valid's auc: 0.954676
Early stopping, best iteration is:
[32]	valid's auc: 0.955083
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.953098
[40]	valid's auc: 0.953042


[I 2025-05-25 21:08:27,520] Trial 1 finished with value: 0.9542268318702188 and parameters: {'num_leaves': 78, 'learning_rate': 0.1720510434701587}. Best is trial 1 with value: 0.9542268318702188.


...

Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.952471
[40]	valid's auc: 0.952814
[60]	valid's auc: 0.953338
[80]	valid's auc: 0.953686
[100]	valid's auc: 0.954054
[120]	valid's auc: 0.95426
[140]	valid's auc: 0.954364
[160]	valid's auc: 0.954462
[180]	valid's auc: 0.954554
[200]	valid's auc: 0.9546
[220]	valid's auc: 0.954723
[240]	valid's auc: 0.954814
[260]	valid's auc: 0.954916
[280]	valid's auc: 0.955086
[300]	valid's auc: 0.95513
[320]	valid's auc: 0.955243
[340]	valid's auc: 0.955531
[360]	valid's auc: 0.955573
[380]	valid's auc: 0.9556
[400]	valid's auc: 0.955584
[420]	valid's auc: 0.955625
[440]	valid's auc: 0.955641
[460]	valid's auc: 0.955693
[480]	valid's auc: 0.955741
[500]	valid's auc: 0.955783
Did not meet early stopping. Best iteration is:
[498]	valid's auc: 0.955785
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.952739
[40]	valid's auc: 0.953256
[60]	valid's auc: 0.953458
[80]	valid's auc: 0.953695
[100

[I 2025-05-25 21:22:11,867] Trial 98 finished with value: 0.9544359395105337 and parameters: {'num_leaves': 40, 'learning_rate': 0.011306367576307626}. Best is trial 62 with value: 0.9545567907832933.


Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.951466
[40]	valid's auc: 0.952843
[60]	valid's auc: 0.953494
[80]	valid's auc: 0.953943
[100]	valid's auc: 0.954118
[120]	valid's auc: 0.954302
[140]	valid's auc: 0.954475
[160]	valid's auc: 0.954709
[180]	valid's auc: 0.954882
[200]	valid's auc: 0.954969
[220]	valid's auc: 0.955214
[240]	valid's auc: 0.955428
[260]	valid's auc: 0.95543
[280]	valid's auc: 0.955477
[300]	valid's auc: 0.955577
[320]	valid's auc: 0.955688
[340]	valid's auc: 0.955745
[360]	valid's auc: 0.955847
[380]	valid's auc: 0.955906
[400]	valid's auc: 0.955972
[420]	valid's auc: 0.95602
[440]	valid's auc: 0.956009
[460]	valid's auc: 0.955993
Early stopping, best iteration is:
[415]	valid's auc: 0.956025
Training until validation scores don't improve for 50 rounds
[20]	valid's auc: 0.952959
[40]	valid's auc: 0.95338
[60]	valid's auc: 0.953718
[80]	valid's auc: 0.953836
[100]	valid's auc: 0.953858
[120]	valid's auc: 0.954166
[140]	valid's 

[I 2025-05-25 21:22:26,294] Trial 99 finished with value: 0.9544943811596447 and parameters: {'num_leaves': 35, 'learning_rate': 0.01691792976812226}. Best is trial 62 with value: 0.9545567907832933.


Лучшие параметры: {'num_leaves': 49, 'learning_rate': 0.02166968919636838}


In [ ]:
lgb_df = lgb.Dataset(data=new_X_train, label=y_train, categorical_feature=['model_4', 'clusters'])

model = lgb.train(
    params={
        'objective': 'binary',
        'num_leaves': 49, 
        'learning_rate': 0.02166968919636838
    },
    train_set=lgb_df,
    num_boost_round=312
)

y_pred = model.predict(test.drop(['order_id', 'order_create_time', 'session_end_time'], axis=1))
pd.DataFrame({'order_id': test.order_id, 'is_callcenter': y_pred}).to_csv('tuned2_submission.csv', index=False)